In [3]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Minimum Spanning Tree: Prim & Kruskal Algorithm


## Spanning Tree -> Minimum Spanning Tree
<br>

<center>
<img src="figures/st0.png"/>
</center>

- We can use BFS, DFS to span a tree for a graph

> For a connected undirected graph $G = (V,E)$, a **spanning tree** is a tree $T = (V,E')$ with $E' \subseteq E$

> **Minimum Spanning Tree (MST)** is for a weighted graph with its minimum weight of T


### Greedy Algorithm -> Light-Edge Property

<br>

Let $G = (V,E,w)$ be a connected undirected, weighted graph with distinct edge weights. A **graph cut** of a graph $(G,V)$ is a partitioning of vertices $V_1 \subset V$, $V_2 = V - V_1$. Each vertex set $V_i \subset V$ defines a **vertex-induced subgraph** consisting of edges where both endpoints are in $V_i$.


<center>
    <img src="figures/mst_example.jpeg"/>
</center>

<br>

- Node Perspective: [Greedily Adding Node -> Prim's Algorithm]
- Edge Perspective [Greedily Adding Edge  -> Kruskal Algorithm]

## Prim's Algorithm

Perform **priority-first search** on $G$ starting from an arbitrary vertex $s$.

To select the next edge to expand the frontier, use priority:
- $p(v) = \min_{x \in X} w(x,v)$
- Add the chosen edge $(u,v)$ to the tree.



<center>
    <img src="figures/prim.jpg" width=40%/>
</center>

- Edge $(c, f)$ has minimum weight across $(X,Y)$.
- So, we visit $f$ by adding it to the frontier


This sounds very similar to Dijkstra's algorithm. What's the difference?

<center>
<img src="figures/subpaths.png" width="40%"/>
</center>


In [ ]:
from heapq import heappush, heappop 

def dijkstra(graph, source):
    def dijkstra_helper(visited, frontier):
        if len(frontier) == 0:
            return visited
        else:
            distance, node = heappop(frontier)
            if node in visited:
                return dijkstra_helper(visited, frontier)
            else:
                print('visiting', node)
                visited[node] = distance
                for neighbor, weight in graph[node]:
                    heappush(frontier, (distance + weight, neighbor))                
                return dijkstra_helper(visited, frontier)
        
    frontier = []
    heappush(frontier, (0, source))
    visited = dict()  # store the final shortest paths for each node.
    return dijkstra_helper(visited, frontier)

graph = {
            's': {('a', 4), ('b', 8)},
            'a': {('s', 4), ('b', 2), ('c', 5)},
            'b': {('s', 8), ('a', 2), ('c', 3)}, 
            'c': {('a', 5), ('b', 3), ('d', 3)},
            'd': {('c', 3)},
        }
dijkstra(graph, 's')

In [ ]:
def prim(graph):
    def prim_helper(visited, frontier, tree):
        if len(frontier) == 0:
            return tree
        else:
            weight, node, parent = heappop(frontier)
            if node in visited:
                return prim_helper(visited, frontier, tree)
            else:
                print('visiting', node)
                # record this edge in the tree
                tree.add((weight, node, parent))
                visited.add(node)
                for neighbor, w in graph[node]:
                    heappush(frontier, (w, neighbor, node)) ## different point ## 
                    # compare with dijkstra:
                    # heappush(frontier, (distance + weight, neighbor))                

                return prim_helper(visited, frontier, tree)
        
    # pick first node as source arbitrarily
    source = list(graph.keys())[0]
    frontier = []
    heappush(frontier, (0, source, source))
    visited = set()  # store the visited nodes (don't need distance anymore)
    tree = set()
    prim_helper(visited, frontier, tree)
    return tree

prim(graph)

## Work of Prim's Algorithm

This does identical work to Dijkstra, so $O(|E| \log |E|)$

Can we just pick an arbitrary source node? Why or why not?

What about directed graphs? Will this work?


No - if source node is not connected to all other nodes.

Even if it is, we may have a suboptimal solution:

![figures/prim-fail.png](figures/prim-fail.png)


## Kruskal Algorithm

Prim's algorithm does a graph search while computing the MST.

Can we just greedily add edges in increasing order of weight?

**Almost**: need to avoid cycles and ensure we "span" the graph


<center>
<img src="figures/kruskal.png" width="80%"/>
</center>





<br>

### Kruskal's Algorithm

“Perform the following step as many times as possible: Among the edges of $G$ not yet chosen, choose the shortest edge which does not form any loops with those edges already chosen.” [Kruskal, 1956]


### Implementing Kruskal's algorithm

"choose the shortest edge which does not form any loops with those edges already chosen"

How do we implement this?

For an edge $(u,v)$, we must check if $u$ and $v$ are in the same connected component, based on the edges added so far.

How can we do this?

We can run BFS or DFS starting at $u$ or $v$, but this would be expensive: $O(|V| + |E|)$ work at each iteration.

<br>

If we think of each connected component as a set of nodes, we need an efficient way of:

- checking which set $u$ and $v$ are in
- determining if these two sets are equal
- if they are not equal, then we need to take their union




To make checking set equality fast, we will assign a **representative** node in each set.

E.g., suppose we have two sets $\mathbf{S} = \{S_1, S_2\}$ where:

$S_1 = \{\mathbf{a},b,c\}~~~~ S_2 = \{\mathbf{s}, d, e\}$

We can (arbitrarily) assign the representative of $S_1$ to be $a$, and the representative of $S_2$ to be $s$.

$r(S_1) = a ~~~ r(S_2) = s$

<br>

If $S_u$ is the set containing $u$ and $S_v$ is the set containing $v$, then we can check if $u$ and $v$ are in the same set by checking if $r(S_u) == r(S_v)$





## Implementation with three operations:

1. `make_set(u)`: create a new set containing the single element $u$
  - $u$ will be the representative of this set
  
2. `find_set(u)`: returns the representative of the set containing $u$:  $r(S_u)$


3. `union(u,v)`: replace $S_u$ and $S_v$ with $S_u \cup S_v$ in the collection of sets $\mathbf{S}$

What data structures can we use to represent each set?

## Data Structure of Balanced Forests

<center>
    <img src="figures/forest.png"/>
</center>

Each set is a balanced tree, where the root is the representative.

Assuming we represent a tree node with a pointer to its parent, what is the work of `find_set(u)` (to find the representative of $S_u$)?

$O(\log n)$, assuming a balanced tree, to walk from a node to its root.

How about `union(u,v)`?


<center>
    <img src="figures/merge.png" width=50%/>
</center>

- find representative of $u ~~~~ O(\log n)$
- find representative of $v ~~~~ O(\log n)$
- link root of one tree to the root of another

<br>

But, how do we ensure tree stays balanced to keep $O(\log n)$ work for `find_set(u)`?


Recall [Leftist Heaps](https://nbviewer.jupyter.org/github/tulane-cmps2200/slides/blob/master/module-06-greedy/greedy-02.ipynb?flush_cache=True) from Greedy module!

<center>
    <img src="figures/meld_schematic.jpg" width="70%">
</center>

similar idea here:

<center>
    <img src="figures/rank.png" width="70%">
</center>

Add "shorter" tree to the "taller" tree.
- store the "rank" of each tree as its depth

<br>

Thus, if $height(S_v) < height(S_u)$, then the height the union of $S_u \cup S_v$ is 

$\max \{ height(S_u), height(S_v)+1\}$

<br>
Using similar arguments as in leftist heaps, we can ensure that height of any tree is $O(\log n)$

### Kruskal's Algorithm

0. Initialize tree $T \leftarrow \emptyset$  

<br>


1. For each $v \in V$, run `make_set(v)`

<br>

2. Sort edges in increasing order of weight

<br>

3. For each edge $e=(u,v)$ in sorted set:
  - if `find_set(u)` $\ne$ `find_set(v)`:
    - $T \leftarrow T \cup \{(u,v)\}$
    - `union(u,v)`


In [5]:
# Python program for Kruskal's algorithm to find
# Minimum Spanning Tree of a given connected,
# undirected and weighted graph

from collections import defaultdict

# Class to represent a graph


class Graph:
    def __init__(self, vertices):
        self.V = vertices # No. of vertices
        self.graph = [] # default dictionary
        # to store graph

    # function to add an edge to graph
    def make_set(self, u, v, w):
        self.graph.append([u, v, w])

    # A utility function to find set of an element i
    # (uses path compression technique)
    def find_set(self, parent, i):
        if parent[i] == i:
            return i
        return self.find_set(parent, parent[i])

    # A function that does union of two sets of x and y
    # (uses union by rank)
    def union(self, parent, rank, x, y):
        xroot = self.find_set(parent, x)
        yroot = self.find_set(parent, y)
        
        # Attach smaller rank tree under root of
        # high rank tree (Union by Rank)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot

            
        # If ranks are same, then make one as root
        # and increment its rank by one
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    # The main function to construct MST using Kruskal's # algorithm
    def KruskalMST(self):
        
        result = [] # This will store the resultant MST
        
        # An index variable, used for sorted edges
        i = 0
        
        # An index variable, used for result[]
        e = 0

        # Step 1: Sort all the edges in non-decreasing order of their weight. 
        # If we are not allowed to change the given graph, we can create a copy of graph
        
        self.graph = sorted(self.graph, key=lambda item: item[2])

        parent = []
        rank = []

        # Create V subsets with single elements
        for node in range(self.V):
            parent.append(node)
            rank.append(0)

        # Number of edges to be taken is equal to V-1
        while e < self.V - 1:

            # Step 2: Pick the smallest edge and increment the index for next iteration
            u, v, w = self.graph[i]
            i = i + 1
            x = self.find_set(parent, u)
            y = self.find_set(parent, v)

            # If including this edge does't cause cycle, include it in result and increment the indexof result for next edge
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.union(parent, rank, x, y)
            # Else discard the edge

        minimumCost = 0
        print ("Edges in the constructed MST")
        for u, v, weight in result:
            minimumCost += weight
            print("%d -- %d == %d" % (u, v, weight))
        print("Minimum Spanning Tree" , minimumCost)

# Driver code
g = Graph(4)
g.make_set(0, 1, 10)
g.make_set(0, 2, 6)
g.make_set(0, 3, 5)
g.make_set(1, 3, 15)
g.make_set(2, 3, 4)

# Function call
g.KruskalMST()


Edges in the constructed MST
2 -- 3 == 4
0 -- 3 == 5
0 -- 1 == 10
Minimum Spanning Tree 19


### Kruskal's Algorithm - work

0. Initialize tree $T \leftarrow \emptyset~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~O(1)$


1. For each $v \in V$, run `make_set(v)` $~~~~~O(|V|)$


2. Sort edges in increasing order of weight $~~O(|E|\lg |E|)$


3. For each edge $e=(u,v)$ in sorted set: $~~O(|E|\lg |V|)$
  - if `find_set(u)` $\ne$ `find_set(v)`: $~~O(\lg |V|)$
    - $T \leftarrow T \cup \{(u,v)\}$
    - `union(u,v)`
    
    
Thus, total work is $O(|E|\lg |E|)$

Since we have $E \in O(|V|^2)$, this is equivalent to:

$O(|E|\lg |V|^2) \equiv O(2|E| \log |V|) \equiv O(|E| \log |V|)$
